In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import *
from pyspark import SparkContext, SparkConf

In [2]:
# create the session
conf = pyspark.SparkConf().setAppName('appName').setMaster('local')
# create the context
sc = pyspark.SparkContext(conf=conf)

In [3]:
spark = SparkSession(sc)

In [4]:
spark

In [6]:
df = spark.read.csv("MoviesA3.csv", sep= ";",inferSchema=True,header=True)

#### 1. Find the title, year and director of action films that won an award.

In [41]:
df.filter((df['Genre'] == "Action") & (df['Awards'] == "Yes")).select('Title', 'Year', 'Director').show()

+-----+----+--------+
|Title|Year|Director|
+-----+----+--------+
+-----+----+--------+



In [65]:
# as the table has no values where (df['Genre'] == "Action") & (df['Awards'] == "Yes") are both true
# we convert set genre of all movies that got an award to Action, so we can see the query works
from pyspark.sql.functions import when
df1 =  df.withColumn("Genre", when(df.Awards == "Yes","Action"))
df1.filter((df1['Genre'] == "Action") & (df1['Awards'] == "Yes")).select('Title', 'Year', 'Director').show()

+--------------------+----+--------------------+
|               Title|Year|            Director|
+--------------------+----+--------------------+
| Fanny and Alexander|1982|     Bergman, Ingmar|
|     A Man & a Woman|1966|     Lelouch, Claude|
|Un Hombre y una M...|1966|     Lelouch, Claude|
| Official Story, The|1985|        Puenzo, Luiz|
|   Wild Strawberries|1957|     Bergman, Ingmar|
|Through a Glass D...|1962|     Bergman, Ingmar|
|    Cries & Whispers|1972|     Bergman, Ingmar|
| Sound of Music, The|1965|        Wise, Robert|
|     Babette's Feast|1987|       Axel, Gabriel|
|Murder on the Ori...|1974|       Lumet, Sidney|
|   Elephant Man, The|1980|        Lynch, David|
|E. T. The Extra-T...|1982|   Spielberg, Steven|
|              Misery|1990|         Reiner, Rob|
|              Gandhi|1982|Attenborough, Ric...|
|       Autumn Sonata|1978|     Bergman, Ingmar|
|            Gaslight|1944|       Cukor, George|
|A Woman Called Golda|1982|        Gibson, Alan|
|           Anastasi

#### 2. For each award-winning actor, find the movies he acted it. Print the names of the movies and the director of the movie.

In [81]:
actors = df.filter((df['Awards'] == "Yes")).select('Actor').distinct()
l = [str(row.Actor) for row in df2.collect()]
df.filter(df.Actor.isin(l)).select('Title', 'Director').show()

+--------------------+----------------+
|               Title|        Director|
+--------------------+----------------+
|                Cuba| Lester, Richard|
|      Days of Heaven|Malick, Terrence|
|     Camille Claudel|  Nuytten, Bruno|
| Fanny and Alexander| Bergman, Ingmar|
|     A Man & a Woman| Lelouch, Claude|
|A Man & a Woman: ...| Lelouch, Claude|
|Un Hombre y una M...| Lelouch, Claude|
| Official Story, The|    Puenzo, Luiz|
|Lindbergh Kidnapp...|     Kulik, Buzz|
|      Donovan's Reef|      Ford, John|
|             Quintet|  Altman, Robert|
|   Wild Strawberries| Bergman, Ingmar|
|   Seventh Seal, The| Bergman, Ingmar|
|              Dreams| Bergman, Ingmar|
|    Naked Night, The| Bergman, Ingmar|
|Through a Glass D...| Bergman, Ingmar|
|    Cries & Whispers| Bergman, Ingmar|
|Barbarian & the G...|    Huston, John|
|              Dr. No|  Young, Terence|
|       Elephant Walk|            null|
+--------------------+----------------+
only showing top 20 rows



#### 3. Find the top 10 most popular movies that did not win an award.

In [96]:
df.filter((df.Awards == 'No')).where(df.Awards.isNotNull()).sort(df.Popularity.desc()).select('Title', 'Popularity').show()

+--------------------+----------+
|               Title|Popularity|
+--------------------+----------+
|        Five Corners|        88|
|        Final Notice|        88|
|Gonzo Presents Mu...|        88|
| Guilty by Suspicion|        88|
|        Swedenhielms|        88|
|           Raw Nerve|        88|
|Happy Birthday to Me|        88|
|         Let It Ride|        88|
|      New Year's Day|        88|
|   Fellini Satyricon|        88|
|   Time Machine, The|        88|
|Long Voyage Home,...|        88|
|           Class Act|        88|
|   Dangerous Pursuit|        88|
|  Best of Times, The|        88|
|Ballad of Narayam...|        88|
|         Head Office|        88|
|     Great Race, The|        88|
|         Terror, The|        88|
|They Were Expendable|        88|
+--------------------+----------+
only showing top 20 rows



#### 4. Find the 10 least popular movies that were released before 1980.

In [98]:
df.filter(df.Year < 1980).where(df.Popularity.isNotNull()).sort(df.Popularity.asc()).select('Year', 'Title', 'Popularity').limit(10).show()

+----+--------------------+----------+
|Year|               Title|Popularity|
+----+--------------------+----------+
|1968|             Shalako|         0|
|1970|             Airport|         0|
|1976|  Shout at the Devil|         0|
|1930|       Anna Christie|         0|
|1953|   Tales of Tomorrow|         0|
|1974|            Stavisky|         1|
|1958|          Indiscreet|         1|
|1971| Anderson Tapes, The|         1|
|1978|           Holocaust|         1|
|1949|Law of the Golden...|         1|
+----+--------------------+----------+



#### 5. Find the average length of the movies of each genre.

In [103]:
df.groupBy("Genre").avg("Length").show(truncate=False)

+---------------+------------------+
|Genre          |avg(Length)       |
+---------------+------------------+
|Crime          |66.0              |
|Romance        |127.0             |
|Adventure      |119.0             |
|null           |120.5             |
|Drama          |113.30455259026688|
|War            |116.90625         |
|Fantasy        |102.0             |
|Mystery        |103.00990099009901|
|Music          |100.48780487804878|
|Science Fiction|106.47368421052632|
|Horror         |93.92727272727272 |
|Short          |40.0              |
|Western        |93.0091743119266  |
|Comedy         |96.50540540540541 |
|Action         |104.5             |
|Westerns       |124.8             |
+---------------+------------------+



#### 6. Find the actor and actress pair who have acted in more than three Comedies together.

In [115]:
import pyspark.sql.functions as f
df.filter((df.Genre == 'Comedy') ).where(df.Actor.isNotNull() & df.Actress.isNotNull()).groupBy('Actor', 'Actress').count().filter(f.col('count') > 2).show()

+--------------+------------------+-----+
|         Actor|           Actress|count|
+--------------+------------------+-----+
|  Allen, Woody|     Keaton, Diane|    5|
|Tracy, Spencer|Hepburn, Katharine|    6|
+--------------+------------------+-----+



#### 7. Find the names of actors who acted in movies of both ‘Comedy’ and ‘Drama’ Genre.

In [117]:
nComedy = df.filter(df.Genre == 'Comedy').select('Actor').distinct()
nDrama = df.filter(df.Genre == 'Drama').select('Actor').distinct()
nComedy.intersect(nDrama).show()

+--------------------+
|               Actor|
+--------------------+
|      Boyer, Charles|
|      Taylor, Robert|
|        Lemmon, Jack|
|       Aiello, Danny|
|      Stewart, James|
|    Malmsten, Birger|
|         Taylor, Rod|
|       Russell, Kurt|
|     Hopkins, Harold|
|      Beatty, Warren|
|       Irons, Jeremy|
|        Fonda, Henry|
|      Ford, Harrison|
|   Depardieu, G�rard|
|     Eastwood, Clint|
|      Howard, Arliss|
|        Cleese, John|
|Lambert, Christopher|
|        Johnson, Ben|
|       Connery, Sean|
+--------------------+
only showing top 20 rows



#### 8. Find the names of actors who acted in movies of both ‘Comedy’ or ‘Drama’ Genre.

In [118]:
nComedy.union(nDrama).show()

+-----------------+
|            Actor|
+-----------------+
|  Matthau, Walter|
|Keillor, Garrison|
|Gauthier, Vincent|
|   Boyer, Charles|
|Finneran, Siohban|
|     Keach, James|
|Nagase, Masatoshi|
|   Taylor, Robert|
|    Mandel, Howie|
|     Chase, Chevy|
|   Wilson, George|
|      Arkin, Alan|
|     Lemmon, Jack|
|Banderas, Antonio|
|     Most, Donald|
|    Jaglom, Henry|
|    Rey, Fernando|
|    Aiello, Danny|
|     Reiner, Carl|
|     Cooper, Gary|
+-----------------+
only showing top 20 rows



#### 9. Find the names of actors who did not acted in any ‘Comedy’

In [119]:
df.select('Actor').distinct().subtract(nComedy).show()

+--------------------+
|               Actor|
+--------------------+
|      Din, Ayub Khan|
|        Ferrer, Jose|
|       Harrison, Rex|
|      McCleery, Gary|
|         Busey, Gary|
|       Kime, Jeffrey|
|     McDowall, Roddy|
|          Race, Hugo|
|Luckinbill, Laurence|
|       Karyo, Tcheky|
|          Webb, John|
|         Ganus, Paul|
|       Longden, John|
|       Fisher, Eddie|
|        Keach, Stacy|
|   Bergen, Robert D.|
|    Sj�str�m, Victor|
|         Morrow, Vic|
|        Singer, Marc|
|    Wilding, Michael|
+--------------------+
only showing top 20 rows



#### 10. Find each actor, find the mean, max, min ranking of his movies.

In [124]:
from pyspark.sql.functions import sum,avg,max,min,mean,count
df.groupBy("Actor").agg(avg("Popularity"), max('Popularity'), min('Popularity')).show(truncate=False)

+--------------------+-----------------+---------------+---------------+
|Actor               |avg(Popularity)  |max(Popularity)|min(Popularity)|
+--------------------+-----------------+---------------+---------------+
|Din, Ayub Khan      |6.0              |6              |6              |
|Ferrer, Jose        |7.0              |7              |7              |
|Harrison, Rex       |10.0             |10             |10             |
|McCleery, Gary      |41.0             |41             |41             |
|Matthau, Walter     |67.33333333333333|83             |52             |
|Keillor, Garrison   |6.0              |6              |6              |
|Busey, Gary         |74.0             |74             |74             |
|Boyer, Charles      |55.0             |70             |25             |
|Kime, Jeffrey       |73.0             |73             |73             |
|Gauthier, Vincent   |11.0             |11             |11             |
|McDowall, Roddy     |55.0             |79         

#### 11. List the number of movies released in each decade starting from 1960’s.

In [135]:
from pyspark.sql.functions import udf
udf_foo = udf(lambda x: int(x/10)*10)
df.filter(df.Year > 1959).withColumn("Year",udf_foo("Year")).groupBy("Year").count().sort(f.col('Year')).show()

+----+-----+
|Year|count|
+----+-----+
|1960|  154|
|1970|  244|
|1980|  607|
|1990|  345|
+----+-----+



#### 12. Find the number of movies released in each year.

In [136]:
df.groupBy("Year").count().sort(f.col('Year')).show()

+----+-----+
|Year|count|
+----+-----+
|1920|    1|
|1923|    1|
|1924|    3|
|1925|    1|
|1926|    4|
|1927|    3|
|1928|    5|
|1929|    5|
|1930|    3|
|1931|    9|
|1932|    8|
|1933|    3|
|1934|    4|
|1935|    8|
|1936|    6|
|1937|    9|
|1938|    8|
|1939|   11|
|1940|   11|
|1941|    7|
+----+-----+
only showing top 20 rows



#### 13. Find the number of movies released in each year of each genre. Consider only the movies with length greater than 100 minutes.

In [146]:
df1 = df.filter(f.col('Length') > 100).groupBy("Year", 'Genre').count()
df1.withColumn('ccol',f.concat(df1['Genre'],f.lit('_cnt'))).groupby('Year').pivot('ccol').agg(f.first('count')).fillna(0).sort(f.col('Year')).show()

+----+----+----------+-------------+----------+---------+-----------+----------+---------+-----------+-----------+-------------------+-------+-----------+------------+
|Year|null|Action_cnt|Adventure_cnt|Comedy_cnt|Drama_cnt|Fantasy_cnt|Horror_cnt|Music_cnt|Mystery_cnt|Romance_cnt|Science Fiction_cnt|War_cnt|Western_cnt|Westerns_cnt|
+----+----+----------+-------------+----------+---------+-----------+----------+---------+-----------+-----------+-------------------+-------+-----------+------------+
|1920|   0|         0|            0|         0|        1|          0|         0|        0|          0|          0|                  0|      0|          0|           0|
|1924|   0|         0|            0|         0|        2|          0|         0|        0|          0|          0|                  0|      0|          0|           0|
|1925|   0|         0|            0|         0|        1|          0|         0|        0|          0|          0|                  0|      0|          0|      

#### 14. Sort the movie’s release before 1990 by the title.

In [147]:
df.filter(f.col('Year') < 1990).select('Year', 'Title').sort(f.col('Title')).show()

+----+--------------------+
|Year|               Title|
+----+--------------------+
|1968|2001: A Space Ody...|
|1982|             48 Hrs.|
|1963|               8 1/2|
|1966|A Big Hand for th...|
|1962|  A Child Is Waiting|
|1985|A Chorus Line, Th...|
|1971|  A Clockwork Orange|
|1967|A Coeur Joie, (He...|
|1988|   A Cry in the Dark|
|1989|  A Dry White Season|
|1966|      A Fine Madness|
|1988| A Fish Called Wanda|
|1964|A Fistful of Dollars|
|1943|     A Guy Named Joe|
|1954|    A Lesson in Love|
|1977|A Little Night Music|
|1966|     A Man & a Woman|
|1986|A Man & a Woman: ...|
|1966|A Man for All Sea...|
|1976|    A Matter of Time|
+----+--------------------+
only showing top 20 rows



#### 15. Find the movies with long titles. A movie title is considered long if it is greater than 50 alphabets.

In [152]:
udf1 = udf(lambda x: len(x))
df.withColumn("Title_length", udf1("Title")).select("Year", "Title", "Title_length" ).filter(f.col('Title_length') > 50).sort(f.col('Title_length').desc()).show()

+----+--------------------+------------+
|Year|               Title|Title_length|
+----+--------------------+------------+
|1979|Fawlty Towers, Go...|          69|
|1990|Industrial Sympho...|          52|
|1992|Unnamable II, The...|          51|
+----+--------------------+------------+



# The End